In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import average_precision_score
import polars as pl

In [12]:
train = pl.read_parquet('data/first_100k.parquet')
X_test_1 = pl.read_parquet('data/second_100k_variables.parquet')
X_test_2 = pl.read_parquet('data/second_100k_target.parquet')
y_test = pl.read_parquet('data/third_100k_variables.parquet')

In [10]:
X_test = pl.concat([X_test_1, X_test_2.select('target')], how='horizontal')

del X_test_1
del X_test_2

In [14]:
train = pl.concat([train, X_test])
train

index,aedbacefbffefeabcfcfbf,afeabcadcadecacbabdacdfefdaeaf,fceaafeddabadcba,edccebcbdcbcdfeecbdadab,acebbabdbeaebfacabeddafafe,dffdbcfdfafedcfbdeafcdbfacd,aceabcadaafcfeaedabfdbabdbcdcaa,fdefedbeedcdccfefbeddf,dbccdcbabbefbccaeefbdaeb,acedbcbefaaeccdacefcabfe,cdbdfaaefafccaedbcdece,ceebebdecbfebdeceabffcafbbedeabd,babffdccdebfcaceefebfdb,bcfeecedfbbdebfcffddad,ebaaeddbbdfefddecfedd,cacadbbcbdcdbbfbdabfcffffda,dbdcbbccedadefabdcbedcdaafddee,ffccddecaeccdbadeccfecb,ecbbafbdbeddaafccddbebbd,badcefdfedabaabeebcfdea,adebbfabcbecefcbeacffaefbb,bebdbbdbcaadfdacab,eedcecddbefcdeadfbdabacecbdfa,bebbfaaeadcedaaacdbcdfafeacf,fdfbccfacdfdabacbfc,badadffcfcfbabefadfa,aaedfcbcabfecdedefa,ffcbafaddbfdaccbdcccecef,faeedacdbbcddfefdbfef,dbeecbfcfccadafcfed,fbcdfdbfbdaddafcdbcffd,ccffaeadfcdcccaabcedfeac,bddaaacddfcabbedeeaeab,aedacbfddfbacaaefeefad,cecdffbeccccdeabfdbecf,ccdcaeeaafffceacafebdbbe,…,ccdbdeaeeaafbdbddaf,acaeedfbdfebadaaddeea,bafdfdaaadecafdaffffdaaca,bccbcfafadeecccf,abaccdfdabebecccfdbccf,bcfcfcdfecfefdbdecbbcdebcc,febbdaeebbbacbefbdefaeffacd,adebbedafcecfbddceefefca,acfdaedefcdceefcbdacdc,ebaccdcaaebffddbadfcddecb,fdecacdaebacfeabffafbeacdb,abfaabeaccafaefdacdaaeecbcaed,eeadceeaffdfdaffadefaadffef,ddbcbefcebefaebbcfbbbeadf,dcdcaaefebdeecabffbeca,ebeddacaeebbffceccbdaabca,cbeadcefbbefbafcfbdaeed,ccaadabdddbaefeccdbcdf,faddccbefedcbcedefebcbbfbdbbd,dcffabddefffbdadacecbeefaad,adeafdfaddfaddefbdad,daaabadadceadeceafbbdf,eaaccccdfcaafdbdeebadab,cfbcfaabdbebaabbebebfbabde,aeccdfaeccaacffdaeffeceaf,dcaaaafbeeebfdecdbad,cebeebadaaeccbbfeaacd,caefcecbeaaedeecaefeccdedcc,fcbbdcffadbdbbabadaf,deebaaaccdececededdbebede,eacacadacadbfeeaddbadfcebbc,aeeefffceecffdafcbadfbb,dfbdccecedcdbfefdefdac,debcabfecffabfcdeacad,beddbdbacbfdeecdcbcedebafdcaf,ebbdcbadddaebdaceafdafecbeacae,target
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,f32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32
1,0.0,503.0,128.0,449.0,54.0,25.0,398.342346,13.217361,398.0,"""329806010756544825""","""18397919539220427995""",0.0,36.0,0.0,1.0,"""15788144898111841938""",1.0,1.0,1.0,1.0,0.0,"""14784228780274844577""",0.0,0.0,0.0,5.065796,1.0,4.0,0.0,4.0,1.0,0.0,2.0,1.0,0.0,2.246237,…,1351.0,1352.0,1256.0,1055.0,83.0,1299.0,1249.0,12437.72168,96.609001,9.0,7.0,2041.0,1745.0,231.0,179.0,274.0,1.77,132.0,128.0,118.0,50.0,172.0,338.0,53.0,1669.0,1351.0,75.0,0.0,0.0,806.0,436403.75,0.012685,0.062425,0.565391,0.091513,0.0,0
2,0.0,69.0,69.0,41.0,28.0,0.0,474.026764,4.026759,474.0,"""17249206226424843767""","""8039259976694302534""",1.0,21.0,1.0,1.0,"""6392996248728262053""",1.0,1.0,1.0,1.0,0.0,"""10540182633566824720""",1.0,0.0,0.0,601.455688,1.0,121.0,3.0,118.0,42.0,0.0,27.0,1.0,15.0,0.220356,…,1386.0,1387.0,1296.0,968.0,76.0,1205.0,1293.0,8662.723633,99.000999,5.0,4.0,1634.0,1679.0,113.0,123.0,308.0,1.45,10.0,10.0,16.0,14.0,87.0,219.0,97.0,1232.0,1390.0,179.0,0.0,0.0,765.0,298147.1875,0.026144,0.400442,0.422764,0.089816,0.0,0
3,1.0,47.0,47.0,27.0,20.0,0.0,439.757568,4.215914,439.0,"""5780749974103040039""","""18397919539220427995""",0.0,29.0,0.0,1.0,"""15760046619303935292""",2.0,1.0,1.0,2.0,1.0,"""275933012517645374""",1.0,0.0,0.0,182.888092,1.0,404.0,1.0,403.0,30.0,0.0,220.0,7.0,7.0,0.020201,…,540.0,540.0,537.0,500.0,64.0,527.0,533.0,1886.021973,96.113998,10.0,8.0,799.0,711.0,69.0,91.0,119.0,1.47,4.0,4.0,0.0,5.0,19.0,57.0,122.0,660.0,540.0,5.0,0.0,0.0,433.0,866752.8125,0.01401,0.071934,0.343354,0.098025,0.0,0
4,0.0,257.0,76.0,127.0,130.0,2.0,1103.71228,2.587257,1103.0,"""10795588512431719735""","""16021691905032725984""",1.0,59.0,0.0,1.0,"""4780749989833272783""",1.0,1.0,0.0,1.0,0.0,"""14784228780274844577""",0.0,0.0,0.0,1265.687988,1.0,232.0,14.0,218.0,112.0,5.0,26.0,1.0,6.0,0.787048,…,3010.0,3010.0,2996.0,2904.0,283.0,2974.0,2961.0,5785.742188,96.530998,5.0,0.0,4459.0,3878.0,566.0,5

In [15]:
new_model = CatBoostClassifier(
    iterations=1_200,
    learning_rate=0.065,
    depth=8,
)

In [16]:
new_model.fit(
    train.select(column for column in train.columns if column != 'target').to_pandas(),
    train.select('target').to_numpy().ravel(),
)

del train

0:	learn: 0.6520431	total: 1.45s	remaining: 28m 59s
1:	learn: 0.6167526	total: 1.95s	remaining: 19m 29s
2:	learn: 0.5869149	total: 2.44s	remaining: 16m 14s
3:	learn: 0.5614145	total: 2.91s	remaining: 14m 30s
4:	learn: 0.5388570	total: 3.49s	remaining: 13m 55s
5:	learn: 0.5196728	total: 4.04s	remaining: 13m 24s
6:	learn: 0.5028633	total: 4.59s	remaining: 13m 3s
7:	learn: 0.4890046	total: 5.05s	remaining: 12m 33s
8:	learn: 0.4770102	total: 5.7s	remaining: 12m 34s
9:	learn: 0.4666869	total: 6.11s	remaining: 12m 7s
10:	learn: 0.4578754	total: 6.57s	remaining: 11m 50s
11:	learn: 0.4500795	total: 7.24s	remaining: 11m 56s
12:	learn: 0.4430255	total: 7.71s	remaining: 11m 44s
13:	learn: 0.4370546	total: 8.18s	remaining: 11m 33s
14:	learn: 0.4320225	total: 8.71s	remaining: 11m 27s
15:	learn: 0.4272709	total: 9.26s	remaining: 11m 25s
16:	learn: 0.4232611	total: 10s	remaining: 11m 35s
17:	learn: 0.4198699	total: 10.9s	remaining: 11m 55s
18:	learn: 0.4168310	total: 11.6s	remaining: 11m 58s
19:	lear

In [31]:
new_predictions = new_model.predict(X_test.to_pandas())

# del X_test

In [1]:
new_score = average_precision_score(y_test.to_numpy().ravel(), new_predictions)
new_score

NameError: name 'average_precision_score' is not defined

In [11]:
def get_score(y_true: pl.DataFrame, y_pred: pl.DataFrame, default_score: float) -> float:
    score = average_precision_score(y_true.to_numpy().ravel(), y_pred.select('prediction').to_numpy().ravel())
    return score/default_score

In [12]:
new_predictions_df = pl.DataFrame(new_predictions, schema=['prediction'])

In [ ]:
default = CatBoostClassifier(verbose=0).fit(
    train.select(column for column in train.columns if column != 'target').to_pandas(),
    train.select('target').to_numpy().ravel(),
)

In [ ]:
default_predictions = default.predict(train.to_pandas())

In [ ]:
default_score = average_precision_score(y_test.to_numpy().ravel(), default_predictions)
default_score

In [15]:
print(f'La nueva nota: {get_score(y_test, new_predictions_df, 0.22236839166738745)}'
)

La nueva nota: 1.0008708148005243


In [16]:
pl.DataFrame(
    {'index': X_test.select('index'), 'prediction': new_predictions_df}
).write_parquet('output/figueroa.parquet')